# Phase 1: Concurrency & Sample Weights

This notebook implements the sample weighting methodology from **MQL5 Article 19850: Label Concurrency**.

## Objectives:
1. Compute concurrent events count (how many labels overlap at each timestamp)
2. Calculate uniqueness weights (corrects for temporal overlap)
3. Calculate return attribution weights (for comparison only)
4. Calculate time decay weights (gives more weight to recent observations)
5. Compare all weighting methods
6. Save weights for model training

## Key Insight from Article:
**Uniqueness weighting** consistently improves model performance by ensuring each observation's influence during training is proportional to its unique information content. This addresses the violation of the IID assumption in financial time series.

**Performance Note**: This notebook uses optimized versions (5-10x faster than standard implementations)

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import optimized functions (5-10x faster than standard versions)
from optimized_concurrent import (
    get_num_conc_events_optimized,
    get_av_uniqueness_from_triple_barrier_optimized
)
from optimized_attribution import (
    get_weights_by_return_optimized,
    get_weights_by_time_decay_optimized
)
from load_data import load_bars

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Libraries imported successfully")
print("✓ Using optimized functions (5-10x performance improvement)")

✓ Libraries imported successfully
✓ Using optimized functions (5-10x performance improvement)


## Step 1: Load Labeled Events

Load the labeled events from `sides.ipynb` that contain:
- **t1**: End time of the label (when barrier was hit)
- **bin**: Binary label (0=timeout/stop loss, 1=profit target)
- **ret**: Return of the trade
- **side**: Position side (1=long, -1=short)

In [2]:
# Load bar data
SYMBOL = 'EURUSD'
BAR_TYPE = 'tick'

print(f"Loading {SYMBOL} {BAR_TYPE} bars...")
df = load_bars(SYMBOL, BAR_TYPE)
close = df['close']

print(f"✓ Loaded {len(df):,} bars")
print(f"  Date range: {df.index[0]} to {df.index[-1]}")
print(f"  Columns: {list(df.columns)}")

INFO: Loading tick bars from: EURUSD_tick_bars_20251101_170526.csv


Loading EURUSD tick bars...


INFO: Loaded 686,033 tick bars
INFO:   Start: 2023-01-02 07:33:51.458001
INFO:   End: 2025-10-31 22:58:59.181001
INFO:   Columns: ['open', 'high', 'low', 'close', 'tick_volume']


✓ Loaded 686,033 bars
  Date range: 2023-01-02 07:33:51.458001 to 2025-10-31 22:58:59.181001
  Columns: ['open', 'high', 'low', 'close', 'tick_volume']


In [3]:
# Load labeled events
# Option 1: Load from saved CSV (if you've saved from sides.ipynb)
events_file = Path('data') / f'{SYMBOL}_triple_barrier_events.csv'

if events_file.exists():
    print(f"Loading events from {events_file}...")
    events = pd.read_csv(events_file, index_col=0, parse_dates=True)
else:
    print("ERROR: Events file not found!")
    print(f"Please save your labeled events from sides.ipynb to: {events_file}")
    print("\nYou can add this to sides.ipynb:")
    print("  events.to_csv('data/EURUSD_labeled_events.csv')")
    raise FileNotFoundError(f"Events file not found: {events_file}")

print(f"\n✓ Loaded {len(events):,} labeled events")
print(f"  Columns: {list(events.columns)}")
print(f"\nLabel distribution:")
print(events['bin'].value_counts(normalize=True).sort_index())

# Display sample
print(f"\nSample events:")
events.head()

Loading events from data\EURUSD_triple_barrier_events.csv...

✓ Loaded 4,377 labeled events
  Columns: ['t1', 'trgt', 'ret', 'bin', 'side']

Label distribution:
bin
0    0.463103
1    0.536897
Name: proportion, dtype: float64

Sample events:


,t1,trgt,ret,bin,side
time,,,,,
2023-01-03 10:04:56.812001,2023-01-03 10:16:13.477001,0.001016,-0.002159,0,1
2023-01-03 10:05:15.772001,2023-01-03 10:17:36.057001,0.001141,-0.001538,0,1
2023-01-03 10:13:48.968001,2023-01-03 10:27:00.805001,0.001753,-0.001228,0,1
2023-01-03 10:18:52.115001,2023-01-03 10:35:15.660001,0.001979,-0.001229,0,1
2023-01-03 10:30:48.800001,2023-01-03 11:02:12.825001,0.001909,-0.001784,0,1


## Step 2: Compute Concurrent Events Count

For each timestamp in our data, count how many labeled events are "active" (their label period overlaps with that timestamp).

**Why this matters**: When 5 events overlap at a timestamp, each event receives 1/5 of the information content at that timestamp. This helps us quantify redundancy in our training data.

In [ ]:
# Compute concurrent events using optimized function
print("Computing concurrent events count...")
print("(Using Numba-optimized version - 5-10x faster)\n")

import time
start_time = time.time()

num_conc_events = get_num_conc_events_optimized(
    close=close,
    label_endtime=events,
    verbose=True
)

elapsed = time.time() - start_time
print(f"\n✓ Computed in {elapsed:.2f} seconds")
print(f"\nConcurrency Statistics:")
print(f"  Mean concurrency: {num_conc_events.mean():.2f}")
print(f"  Max concurrency: {num_conc_events.max():.0f}")
print(f"  Median concurrency: {num_conc_events.median():.2f}")
print(f"  Min concurrency: {num_conc_events.min():.0f}")

Computing concurrent events count...
(Using Numba-optimized version - 5-10x faster)



TypeError: get_num_conc_events_optimized() got an unexpected keyword argument 'events'

In [ ]:
# Visualize concurrency over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Time series plot
axes[0].plot(num_conc_events.index, num_conc_events.values, linewidth=0.5, alpha=0.7)
axes[0].axhline(num_conc_events.mean(), color='r', linestyle='--', label=f'Mean: {num_conc_events.mean():.2f}')
axes[0].set_title('Number of Concurrent Events Over Time', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Concurrent Events')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution histogram
axes[1].hist(num_conc_events.values, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(num_conc_events.mean(), color='r', linestyle='--', label=f'Mean: {num_conc_events.mean():.2f}')
axes[1].set_title('Distribution of Concurrent Events', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Number of Concurrent Events')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/concurrent_events.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nInterpretation:")
print(f"On average, {num_conc_events.mean():.1f} labels are active at any given timestamp.")
print(f"This means each observation shares information with ~{num_conc_events.mean():.0f} other labels.")
print(f"Without weighting, this redundancy causes overfitting!")

## Step 3: Compute Uniqueness Weights

Calculate the **average uniqueness** for each event:
- For each bar during an event's lifespan, compute `1 / concurrency_at_that_bar`
- Average these values across the event's lifespan
- Result: Events with low overlap get weight ~1.0, highly overlapping events get weight ~0.2-0.4

**Article Finding**: This method improved F1 score by 6.7% (Bollinger) and 10.2% (MA Crossover)

In [ ]:
# Calculate uniqueness weights using optimized function
print("Computing uniqueness weights...")
print("(Using Numba-optimized version - 3-5x faster)\n")

start_time = time.time()

uniqueness_weights = get_av_uniqueness_from_triple_barrier_optimized(
    triple_barrier_events=events,
    close_series=close,
    num_conc_events=num_conc_events,
    verbose=True
)

elapsed = time.time() - start_time
print(f"\n✓ Computed in {elapsed:.2f} seconds")
print(f"\nUniqueness Weight Statistics:")
print(uniqueness_weights['tW'].describe())

# Store in events DataFrame
events['uniqueness'] = uniqueness_weights['tW']

In [ ]:
# Visualize uniqueness weights
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series
axes[0, 0].plot(uniqueness_weights.index, uniqueness_weights['tW'].values, linewidth=0.5, alpha=0.7)
axes[0, 0].axhline(uniqueness_weights['tW'].mean(), color='r', linestyle='--', 
                   label=f'Mean: {uniqueness_weights["tW"].mean():.3f}')
axes[0, 0].set_title('Uniqueness Weights Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Uniqueness Weight')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Distribution
axes[0, 1].hist(uniqueness_weights['tW'].values, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(uniqueness_weights['tW'].mean(), color='r', linestyle='--', 
                   label=f'Mean: {uniqueness_weights["tW"].mean():.3f}')
axes[0, 1].set_title('Distribution of Uniqueness Weights', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Uniqueness Weight')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Scatter: Concurrency vs Uniqueness
sample_indices = events.index.intersection(num_conc_events.index)
conc_at_start = num_conc_events.loc[sample_indices]
unique_vals = uniqueness_weights.loc[sample_indices, 'tW']

axes[1, 0].scatter(conc_at_start, unique_vals, alpha=0.3, s=10)
axes[1, 0].set_title('Uniqueness vs Concurrency at Event Start', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Concurrent Events')
axes[1, 0].set_ylabel('Uniqueness Weight')
axes[1, 0].grid(True, alpha=0.3)

# Box plot by bins
bins = pd.cut(uniqueness_weights['tW'], bins=5)
bin_data = [uniqueness_weights['tW'][bins == b].values for b in bins.cat.categories]
axes[1, 1].boxplot(bin_data, labels=[f'{b.left:.2f}-{b.right:.2f}' for b in bins.cat.categories])
axes[1, 1].set_title('Uniqueness Weight Distribution by Bins', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Weight Range')
axes[1, 1].set_ylabel('Uniqueness Weight')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/uniqueness_weights.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nInterpretation:")
print(f"Average uniqueness: {uniqueness_weights['tW'].mean():.3f}")
print(f"This will be used as max_samples in BaggingClassifier")
print(f"\nWeight interpretation:")
print(f"  1.0 = Completely unique (no overlap)")
print(f"  0.5 = Moderate overlap")
print(f"  <0.3 = Heavy overlap (mostly redundant information)")

## Step 4: Compute Return Attribution Weights

Calculate weights based on absolute returns during each event's lifespan.

**⚠️ WARNING FROM ARTICLE**: This method caused model collapse in meta-labeling experiments. It's included here for comparison purposes only. **Do NOT use for production models.**

In [ ]:
# Calculate return attribution weights using optimized function
print("Computing return attribution weights...")
print("⚠️  WARNING: Article shows this method can cause model collapse!")
print("   Use only for comparison, not production.\n")

start_time = time.time()

return_weights = get_weights_by_return_optimized(
    triple_barrier_events=events,
    close_series=close,
    num_conc_events=num_conc_events,
    verbose=True
)

elapsed = time.time() - start_time
print(f"\n✓ Computed in {elapsed:.2f} seconds")
print(f"\nReturn Attribution Weight Statistics:")
print(return_weights.describe())

# Store in events DataFrame
events['return_attr'] = return_weights

In [ ]:
# Visualize return attribution weights
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
axes[0].hist(return_weights.values, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(return_weights.mean(), color='r', linestyle='--', 
                label=f'Mean: {return_weights.mean():.3f}')
axes[0].set_title('Distribution of Return Attribution Weights', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Weight')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Time series
axes[1].plot(return_weights.index, return_weights.values, linewidth=0.5, alpha=0.7)
axes[1].axhline(return_weights.mean(), color='r', linestyle='--', 
                label=f'Mean: {return_weights.mean():.3f}')
axes[1].set_title('Return Attribution Weights Over Time', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Weight')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/return_attribution_weights.png', dpi=300, bbox_inches='tight')
plt.show()

## Step 5: Compute Time Decay Weights

Apply time decay to give more weight to recent observations.

**Note**: Decay is based on cumulative uniqueness (not chronological time) to avoid reducing weights too fast when observations are redundant.

In [ ]:
# Calculate time decay weights using optimized function
print("Computing time decay weights...")
print("(Using exponential decay with last_weight=0.5)\n")

start_time = time.time()

time_decay_weights = get_weights_by_time_decay_optimized(
    triple_barrier_events=events,
    close_series=close,
    last_weight=0.5,  # Most recent gets 1.0, oldest gets 0.5
    linear=False,  # Use exponential decay
    av_uniqueness=uniqueness_weights,
    verbose=True
)

elapsed = time.time() - start_time
print(f"\n✓ Computed in {elapsed:.2f} seconds")
print(f"\nTime Decay Weight Statistics:")
print(time_decay_weights.describe())

# Store in events DataFrame
events['time_decay'] = time_decay_weights

In [ ]:
# Visualize time decay pattern
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time series showing decay
axes[0].plot(time_decay_weights.index, time_decay_weights.values, linewidth=1, alpha=0.8)
axes[0].set_title('Time Decay Weights (Exponential)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Weight')
axes[0].set_xlabel('Time')
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(time_decay_weights.values, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(time_decay_weights.mean(), color='r', linestyle='--', 
                label=f'Mean: {time_decay_weights.mean():.3f}')
axes[1].set_title('Distribution of Time Decay Weights', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Weight')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/time_decay_weights.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nInterpretation:")
print(f"Most recent observations have weight ~{time_decay_weights.iloc[-100:].mean():.3f}")
print(f"Oldest observations have weight ~{time_decay_weights.iloc[:100].mean():.3f}")
print(f"Decay factor: {time_decay_weights.iloc[:100].mean() / time_decay_weights.iloc[-100:].mean():.2f}x")

## Step 6: Compare All Weighting Methods

Create combined weights and analyze correlations between different weighting schemes.

In [ ]:
# Create combined weights (uniqueness × time_decay)
events['combined'] = events['uniqueness'] * events['time_decay']

# Normalize combined weights so they sum to number of samples
events['combined'] = events['combined'] * len(events) / events['combined'].sum()

print("Weight Correlations:")
weight_cols = ['uniqueness', 'return_attr', 'time_decay', 'combined']
correlation_matrix = events[weight_cols].corr()
print(correlation_matrix.round(3))

# Visualize correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Between Weighting Methods', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('results/weight_correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Visualize all weighting methods side-by-side
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

weight_methods = {
    'uniqueness': 'Uniqueness\n(RECOMMENDED)',
    'return_attr': 'Return Attribution\n(⚠️ Can cause collapse)',
    'time_decay': 'Time Decay',
    'combined': 'Combined\n(Uniqueness × Time Decay)'
}

for idx, (col, title) in enumerate(weight_methods.items()):
    ax = axes[idx // 2, idx % 2]
    
    # Histogram
    ax.hist(events[col].values, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(events[col].mean(), color='r', linestyle='--', linewidth=2,
               label=f'Mean: {events[col].mean():.3f}')
    ax.axvline(events[col].median(), color='g', linestyle='--', linewidth=2,
               label=f'Median: {events[col].median():.3f}')
    
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.set_xlabel('Weight')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/all_weights_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Summary statistics table
print("\n" + "="*80)
print("SUMMARY STATISTICS FOR ALL WEIGHTING METHODS")
print("="*80)

summary_stats = events[weight_cols].describe().T
summary_stats['range'] = summary_stats['max'] - summary_stats['min']
summary_stats['cv'] = summary_stats['std'] / summary_stats['mean']  # Coefficient of variation

print(summary_stats.round(4))

print("\n" + "="*80)
print("RECOMMENDATIONS FROM ARTICLE 19850:")
print("="*80)
print("✓ RECOMMENDED: Use 'uniqueness' or 'combined' for model training")
print("✓ Set max_samples in BaggingClassifier to:", f"{events['uniqueness'].mean():.3f}")
print("✗ AVOID: 'return_attr' caused model collapse in article experiments")
print("="*80)

## Step 7: Save Weights for Model Training

Save all computed weights to CSV for use in Phase 2 (models.ipynb).

In [ ]:
# Create data directory if it doesn't exist
Path('data').mkdir(exist_ok=True)

# Save weights
weights_file = Path('data') / f'{SYMBOL}_sample_weights.csv'
events[weight_cols].to_csv(weights_file)

print(f"✓ Saved sample weights to: {weights_file}")
print(f"  Shape: {events[weight_cols].shape}")
print(f"  Columns: {weight_cols}")

# Also save events with weights for convenience
events_with_weights_file = Path('data') / f'{SYMBOL}_events_with_weights.csv'
events.to_csv(events_with_weights_file)
print(f"\n✓ Saved events with weights to: {events_with_weights_file}")

# Save concurrent events for future reference
conc_file = Path('data') / f'{SYMBOL}_concurrent_events.csv'
num_conc_events.to_csv(conc_file)
print(f"✓ Saved concurrent events to: {conc_file}")

## Summary & Next Steps

### What We Accomplished:
1. ✓ Computed concurrent events count (quantified label overlap)
2. ✓ Calculated uniqueness weights (corrects for temporal redundancy)
3. ✓ Calculated return attribution weights (for comparison)
4. ✓ Calculated time decay weights (emphasizes recent data)
5. ✓ Created combined weights (uniqueness × time_decay)
6. ✓ Saved all weights for model training

### Key Findings:
- Average concurrency: Shows typical label overlap
- Average uniqueness: Will be used as `max_samples` parameter
- Weight correlations: Understand relationships between methods

### Next Phase:
**Phase 2: Model Training** (models.ipynb)
- Implement PurgedKFold cross-validation
- Train Random Forest with sample weights
- Use BaggingClassifier with constrained samples
- Compare weighted vs unweighted models

### Article Citation:
Based on: **Machine Learning Blueprint (Part 4): The Hidden Flaw in Your Financial ML Pipeline — Label Concurrency**  
https://www.mql5.com/en/articles/19850

In [ ]:
# Final summary
print("\n" + "="*80)
print("PHASE 1 COMPLETE: CONCURRENCY & SAMPLE WEIGHTS")
print("="*80)
print(f"\nDataset: {SYMBOL} {BAR_TYPE} bars")
print(f"Total events: {len(events):,}")
print(f"Date range: {events.index[0]} to {events.index[-1]}")
print(f"\nConcurrency:")
print(f"  Mean: {num_conc_events.mean():.2f}")
print(f"  Max: {num_conc_events.max():.0f}")
print(f"\nRecommended Parameters for Phase 2:")
print(f"  max_samples (BaggingClassifier): {events['uniqueness'].mean():.3f}")
print(f"  sample_weight: Use 'uniqueness' or 'combined' column")
print(f"\nFiles saved:")
print(f"  ✓ {weights_file}")
print(f"  ✓ {events_with_weights_file}")
print(f"  ✓ {conc_file}")
print(f"\nReady for Phase 2: Model Training!")
print("="*80)